# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(api_key='sk-50CS5aC9XCnMshTI18840fDc3bB3480082418e61064c7c88',
    base_url='https://api.xiaoai.plus/v1' )

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图片展示了一条穿越郁郁葱葱的草地的木板路。图片中，天空呈现出美丽的蓝色和白色云朵，草地覆盖着鲜绿色的植被。木板路直线延伸，在草丛中开辟出一条通道，引导视线深入画面。这样的场景通常给人一种平静和放松的感觉，可能是一个适合散步或自然观察的地方。整个环境透露出自然和未被破坏的美感，非常适合欣赏户外风光和享受宁静时刻。', role='assistant', function_call=None, tool_calls=None))


In [4]:
response.choices[0].message.content

'这幅图片展示了一条穿越郁郁葱葱的草地的木板路。图片中，天空呈现出美丽的蓝色和白色云朵，草地覆盖着鲜绿色的植被。木板路直线延伸，在草丛中开辟出一条通道，引导视线深入画面。这样的场景通常给人一种平静和放松的感觉，可能是一个适合散步或自然观察的地方。整个环境透露出自然和未被破坏的美感，非常适合欣赏户外风光和享受宁静时刻。'

### 封装成一个函数 query_image_description

In [5]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(api_key='sk-50CS5aC9XCnMshTI18840fDc3bB3480082418e61064c7c88',
    base_url='https://api.xiaoai.plus/v1' )  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [8]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这是一幅搞笑的图片，用以展现两只喜悦犬（Shiba Inu）的对比图。左边的狗被赋予了夸张的人类健身者的身体，图标说明这是“16岁的我”，配文有“我刚洗个澡”，“奖我一口奶酪”，“身体棒棒哒”和“人心叵测的大都市”。

右边的狗则显得更为普通和沮丧，标识为“工作后的我”，配文包括“好累脑子晕”，“好想躺懒椅”等，表达了一种工作后的疲惫与无力感。

整个图片通过幽默的方式表达了生活中工作与年轻时的对比，展示了日常压力与岁月变化对一个人（或在这里是狗）的影响。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [1]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(base_url='https://api.xiaoai.plus/v1' )  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图片中的内容", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [2]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图表展示了从1980年到2020年美国、中国、日本和德国的国内生产总值（GDP）比较。横坐标表示年份，构成40年的时间跨度，纵坐标表示国内生产总值，单位是万亿美元。图中的不同颜色代表不同的国家：蓝色代表美国，红色代表中国，紫色代表日本，绿色代表德国。

通过这个图表，我们可以观察到以下几点：
1. **美国的GDP较高，增长趋势平稳上升。**
2. **中国的GDP从1990年代中期开始显著增长，尤其在2000年之后增速非常快，逐渐接近甚至可能超越美国。**
3. **日本的GDP在1990年代达到顶峰后有所波动，但整体而言较为平稳。**
4. **德国的GDP增长较为稳定，但总体规模小于美国和中国。**

这些数据有助于分析不同国家经济的发展趋势和世界经济格局的变化。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [12]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片是一本笔记本的一页，内容涉及深度学习相关的一些高级技术，特别是与自然语言处理和Transformer模型有关的调优方法。主要内容包括：

1. **Prompt Tuning (FMT + Small Model)**：这是一种只调整模型输入部分（即提示部分，例如模型的开始输入）的方法。在这里并没有具体解释FMT是什么，但通常这指的是一些模型的输入处理方式。

2. **Prefix Tuning**：在训练Transformer模型时，一种不调整整个模型而只调整输入序列前缀的技术，这可以帮助模型在处理特定类型数据时表现更好。

3. **LoRA（Low-Rank Adaptation）技术简介**：这部分介绍了LoRA技术，即通过低秩矩阵分解来适应或调整Transformer模型的权重（W），从而不必对整个权重矩阵进行大规模修改，而只需更新一个较小的ΔW矩阵。该部分还提到了关于ΔW的计算方法，该方法是通过两个较小的矩阵A和B乘积来近似实现。

4. **模型容量和空间消耗**：文中提到LoRA技术的存储要求，比如LAMA使用LoRA时大约是78GB，而Quora使用LoRA时大约是48GB。这显示了LoRA技术在处理大型数据集时相对较小的存储需求。

这些内容显示了笔记的作者在探究如何有效地使用较少的资源来调整和优化深度学习模型，特别是在自然语言处理领域。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [11]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。

![](./images/handwriting_1.jpg)

In [7]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

NameError: name 'Markdown' is not defined

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。
content = query_base64_image_description("./images/math.png")
print(content)

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [20]:
##第一个作业:

![](./images/math.png)

In [3]:
content = query_base64_image_description("./images/math.png","解释一下图片中的数学题和答得数学内容")
print(content)

这张图片包含了几个数学问题及其解法，下面我将逐个解释:

1. 首部描述的是一个关于绝对值和二次方程的不等式问题。其中涉及的公式是 |ab-a|≥0, (b-1)^2=0, 由此导出 b=1, 但解析不完全，需要更多上下文来完全解决。

2. 紧接着，展示了一个关于数列求和的问题，求和公式为从 x_1+x_2+x_3+...+x_206=x_206+1 + x_206+2+...+x_206+5, 相关于算术级数和级数的计算。

3. 其次，提到了使用符号计算来解决系列等式问题，如 2=2×1, 2+4=6=2×3, 2+4+6=12=3×4, 2+4+6+8=20=4×5，这构成了一个关于等差数列求和的典型例子。

4. 然后，讨论了幂的计算——包括找出满足特定条件的幂指数关系式。这一段内容考察如何使用指数法则来简化和求解幂运算相关的等式。

5. 最后，底部列出的题目可能是对学生的一些具体的数学练习题，包括基础的代数运算和一个关于序列求和的练习，如 a_n=6d, 2+4+6+8+...+42, 典型的等差数列求和问题。

以上总结了图片中的主要数学内容和问题。要完全解答这些数学问题，需要具体的条件和更多的数学工具和原理。


In [ ]:
回答:这张图片包含了几个数学问题及其解法，下面我将逐个解释:

1. 首部描述的是一个关于绝对值和二次方程的不等式问题。其中涉及的公式是 |ab-a|≥0, (b-1)^2=0, 由此导出 b=1, 但解析不完全，需要更多上下文来完全解决。

2. 紧接着，展示了一个关于数列求和的问题，求和公式为从 x_1+x_2+x_3+...+x_206=x_206+1 + x_206+2+...+x_206+5, 相关于算术级数和级数的计算。

3. 其次，提到了使用符号计算来解决系列等式问题，如 2=2×1, 2+4=6=2×3, 2+4+6=12=3×4, 2+4+6+8=20=4×5，这构成了一个关于等差数列求和的典型例子。

4. 然后，讨论了幂的计算——包括找出满足特定条件的幂指数关系式。这一段内容考察如何使用指数法则来简化和求解幂运算相关的等式。

5. 最后，底部列出的题目可能是对学生的一些具体的数学练习题，包括基础的代数运算和一个关于序列求和的练习，如 a_n=6d, 2+4+6+8+...+42, 典型的等差数列求和问题。

以上总结了图片中的主要数学内容和问题。要完全解答这些数学问题，需要具体的条件和更多的数学工具和原理。

In [ ]:
##第二个作业:

In [2]:
from IPython.display import display, Markdown
content = query_base64_image_description("./images/math.png","解释一下图片中的数学题和答得数学内容")
display(Markdown(content))

这是一段数学教材或提纲，涵盖了几个不同的数学概念和问题，主要包括了代数表达式、多项式等式以及他们的运算法则。下面简要解释图片中的内容：

1. **等式解析与多项式求和**:
   - 开头部分给出了复杂的式子，例如 `(ab-2)^2=0, (b-1)^2=0, bf(ka)=2-0-b`，这些是展示代数运算和变形。
   - 给出了序列求和的公式：例如`1^3+2^3+...+(n-5)^3+(n-2)^3=200x_n`，实际上是要求序列的立方和。

2. **示例计算**:
   - 演示了简单的加法和乘法如 `2+2=2×2`，`2+4=2×3` 以及 `2+4+6=2×3`。
   - 这些都是为了演示加法如何与乘法对应，很可能是为了教授初级数学概念。

3. **乘法规律问题**:
   - 提出了解决这种特定形式的数学问题的方法，例如，对于 n>=6 时, `2+4+6+...+2n=42`，和`n(n+1)=6⨉k+1⨉6^k+2`的求解方式。

4. **幂运算**:
   - 展示了不同的幂运算问题，如`3^2=1×8、5^2=8×2`等，这有助于学生理解幂的基本概念和操作。

5. **不等式和方程求解**:
   - 也提及了一些方程和不等式的求解方法，示例如何设置和解决这些类型的数学问题。

图片中的内容是对一系列基本数学概念的介绍和练习题，适合用于学生的学习和巩固相关的数学知识。

In [ ]:
回答:这是一段数学教材或提纲，涵盖了几个不同的数学概念和问题，主要包括了代数表达式、多项式等式以及他们的运算法则。下面简要解释图片中的内容：

等式解析与多项式求和:

开头部分给出了复杂的式子，例如 (ab-2)^2=0, (b-1)^2=0, bf(ka)=2-0-b，这些是展示代数运算和变形。
给出了序列求和的公式：例如1^3+2^3+...+(n-5)^3+(n-2)^3=200x_n，实际上是要求序列的立方和。
示例计算:

演示了简单的加法和乘法如 2+2=2×2，2+4=2×3 以及 2+4+6=2×3。
这些都是为了演示加法如何与乘法对应，很可能是为了教授初级数学概念。
乘法规律问题:

提出了解决这种特定形式的数学问题的方法，例如，对于 n>=6 时, 2+4+6+...+2n=42，和n(n+1)=6⨉k+1⨉6^k+2的求解方式。
幂运算:

展示了不同的幂运算问题，如3^2=1×8、5^2=8×2等，这有助于学生理解幂的基本概念和操作。
不等式和方程求解:

也提及了一些方程和不等式的求解方法，示例如何设置和解决这些类型的数学问题。
图片中的内容是对一系列基本数学概念的介绍和练习题，适合用于学生的学习和巩固相关的数学知识。